In [1]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import time

In [47]:
working_dir = "/home/fvalle/phd/datasets/crectal"
os.chdir(working_dir)
dirs = os.listdir("data")

In [48]:
len(dirs)

522

In [50]:
df_genes = pd.read_csv("genes.txt", index_col=[0])
protein_coding = df_genes[df_genes['type_of_gene']=='protein-coding']
#mirna = df_genes[df_genes['type_of_gene']=='microRNA']
select_genes = protein_coding.index.values
#select_genes = mirna.index.values

In [51]:
df = pd.read_csv(("%s/genes.txt"%working_dir), header=[0], index_col=[0])
#print(len(df['gene']))
df = df.loc[select_genes,:]
df.head()

,lenght,type_of_gene,exons
ENSG00000167578,18722.0,protein-coding,18696.0
ENSG00000078237,46830.0,protein-coding,38812.0
ENSG00000146083,11328.0,protein-coding,10691.0
ENSG00000158486,226330.0,protein-coding,217697.0
ENSG00000198242,4966.0,protein-coding,4375.0


In [38]:
#fpkm
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM]{5}.txt[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [39]:
#counts
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.htseq]{6}[\.\_counts]{7}[\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile
    print("Nothing found")
    print(os.listdir(directory))

In [ ]:
#fpkm-uq
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM-UQ]{0,8}[\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [ ]:
##miRNA
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.mirbase21\.mirnas\.quantification][\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [52]:
maxacceptables = 15000
added = 0
for i,cdirectory in enumerate(dirs):
    if re.match("[manifest\.txt]",cdirectory):
        continue
    if "Icon" in cdirectory:
        continue
    cfile = getFilenameFromDir("data/%s"%cdirectory)
    #put header=0 for miRNA!
    #cdf = pd.read_csv(("%s/data/%s/%s"%(working_dir,cdirectory,cfile)), sep='\t', header=0)
    cdf = pd.read_csv(("%s/data/%s/%s"%(working_dir,cdirectory,cfile)), sep='\t')
    #cdf.drop(labels=["reads_per_million_miRNA_mapped","cross-mapped"], axis=1, inplace=True) #miRNA only
    cdf.columns = ["gene", cfile[:]]
    cdf['gene'] = [gene[:15] for gene in cdf['gene']]
    cdf.set_index('gene',inplace=True)
    try:
        df.insert(1,cdf.keys()[0][:],cdf.loc[select_genes,cdf.keys()[0]].values[:60483])
        #df.insert(0,cdf.keys()[0][:],cdf.values.T[0]) #miRNA are not protein-coding
        added=added+1
    except:
        print(sys.exc_info()[0])
    if added >= maxacceptables:
        break
print(added, i)

b1868d9d-ef06-4353-883b-230dae23f8b6.htseq.counts.gz
564daa81-cfef-45b6-94a0-3249b2724d9b.htseq.counts.gz
f1fcd165-ddbc-44b8-b062-ae5199c53cba.htseq.counts.gz
c4e074f2-e21d-4a1a-acf3-b7916f11fb43.htseq.counts.gz
f590941d-19dc-427a-95b6-942c97ea8333.htseq.counts.gz
424d8e5f-9fc6-470b-ad2c-b4447b0eb07e.htseq.counts.gz
82e00e45-734c-471f-ba97-79ec3b7e0baa.htseq.counts.gz
caaf2201-e0d8-46af-9d53-ae776306761f.htseq.counts.gz
902732ab-8a29-46ae-a06d-556ab3cb6825.htseq.counts.gz
e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
7f5a924a-ddf3-45ff-be1f-5b5909305f46.htseq.counts.gz
2f69766d-36f5-446d-ac2f-edf68ce4eaa9.htseq.counts.gz
5e2b0dc5-c09b-4334-9f2f-c8e75380d45c.htseq.counts.gz
1cbb39ca-efbf-4080-bd32-1b76006394a8.htseq.counts.gz
e0d6a42a-2efb-4b3f-9b59-da07dadc01a4.htseq.counts.gz
e875ae4e-4645-4e84-b0ff-9c9a694717a9.htseq.counts.gz
52e5f11b-9a73-4535-9e4d-720fe8feca0a.htseq.counts.gz
562e2cb3-078f-4796-b8a2-a0196713ebac.htseq.counts.gz
d2587070-cb7d-440d-ae49-52f5077248e6.htseq.cou

97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz
9daee176-d523-4450-8ac8-0f69d028399e.htseq.counts.gz
020aa019-a3a4-4055-92ee-be824a597501.htseq.counts.gz
988eba37-1b57-4740-b0ba-9491b57421ab.htseq.counts.gz
d5607680-758b-4ee5-ae0f-a61f7c3133c1.htseq.counts.gz
15864159-be88-41c8-bdef-c2c5927cb1a1.htseq.counts.gz
d265829a-9e49-4e21-ba72-67f5ba406204.htseq.counts.gz
fda5b844-1fa4-462e-bd3b-482025f6ee30.htseq.counts.gz
2a2b0e0f-d54a-4902-8778-500364fcc0c5.htseq.counts.gz
737a6ce7-5446-4610-bf9f-5c46b769b842.htseq.counts.gz
f5c9ea63-ff66-441a-9a6b-055fdcd7cae9.htseq.counts.gz
8bc10161-4cca-46c5-985e-e07349484ec2.htseq.counts.gz
67d1e5e4-91de-43df-b127-ffeef6070603.htseq.counts.gz
7f9cdd9b-cba2-4847-82e9-4e1df1cd9857.htseq.counts.gz
bd8839a3-4dcb-4688-b051-6f09c18d7fc7.htseq.counts.gz
751f84dc-7c2d-417a-bc1f-6d5771f6f09f.htseq.counts.gz
a902f09a-115f-4374-ad8a-4aace335fbbb.htseq.counts.gz
9ab691dc-cbc9-4bc5-8bd9-865783762473.htseq.counts.gz
e2bdca76-7bc5-4143-a6f6-3c6642425c65.htseq.cou

00cc9b4d-a847-464e-979a-7751e1a87ae3.htseq.counts.gz
9a62fe1f-36ec-4e8e-b3d9-bdfc62f71905.htseq.counts.gz
e39de159-abd7-4185-8c20-ca6f27827f31.htseq.counts.gz
fdd24a0e-f3ef-4a5a-87cf-f99884b90fb9.htseq.counts.gz
c2b0ac40-50ff-47e1-8089-c920a932ed9b.htseq.counts.gz
08239d16-937c-4758-9e3d-38217e4c9f35.htseq.counts.gz
ee926a57-93f9-4eb8-86da-224f5daac92a.htseq.counts.gz
6e2db920-dc3e-4a74-8cdb-470cb61e2fac.htseq.counts.gz
2c09404f-c5e8-4e8a-a51b-c8bdbc923efe.htseq.counts.gz
72575c62-b2a8-4fde-8b05-ef6d76df0462.htseq.counts.gz
bb5ddfe6-cd9a-4702-873c-9607e6522161.htseq.counts.gz
9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
dd9a3ab6-6c93-4b47-9b4e-62e56767054b.htseq.counts.gz
ee4dcccc-514b-4cc6-ae63-6ed3e7519a40.htseq.counts.gz
e158f016-7605-45b7-8e4f-bbe313fc461c.htseq.counts.gz
1106a73f-5abd-4a2e-9e07-86ac7a737b77.htseq.counts.gz
514976b1-4fb9-4924-8bbf-ffbf5d7eaad5.htseq.counts.gz
1f5ff5c6-64a8-4214-b304-634aae40202a.htseq.counts.gz
4ea4ed16-8597-426c-9485-0fc8a8d81973.htseq.cou

In [41]:
#df = pd.read_csv(("%s/mainTable.csv"%working_dir))

In [42]:
#df.columns=['gene', 'breast', 'kidney', 'bronchus', 'brain']

In [53]:
df.drop('type_of_gene', inplace=True, axis=1)
df.drop('lenght', inplace=True, axis=1)
df.drop('exons', inplace=True, axis=1)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19369 entries, ENSG00000167578 to ENSG00000181518
Columns: 407 entries, f38b2f21-c5a5-4699-9727-eae9c96aac55.htseq.counts.gz to b1868d9d-ef06-4353-883b-230dae23f8b6.htseq.counts.gz
dtypes: float64(407)
memory usage: 60.3+ MB


In [55]:
print(("genes:%d\trealizations:%d"%(len(df.index),len(df.columns))))

genes:19369	realizations:407


In [56]:
df.dropna().astype(int).to_csv(("%s/mainTable.csv"%working_dir), index=True)

expression per gene

In [ ]:
np.sum(df.loc[0,:].values[1:])

expression per case

In [ ]:
np.sum(df.loc[:,df.keys()[1]].values)

In [ ]:
df_files=pd.read_csv("files.dat",header=0,index_col=0)
df_files[df_files.index.isin(df.columns)].to_csv("files.dat",index=True,header=True)

## Zipf

In [ ]:
A = [df.loc[:,df.keys()[9]].values, df.loc[:,df.keys()[45]].values, df.loc[:,df.keys()[42]].values]
colors = ['red', 'blue', 'yellow']
fig = plt.figure()
x = np.arange(1, len(A[0]))
plt.ylabel("frequencyi")
plt.xlabel("RANKi")
plt.yscale('log')
plt.xscale('log')
plt.plot(x, 1./x, 'g--', label='$r^{-\\alpha}$')
for i,a in enumerate(A):
    plt.plot(np.sort(a/np.sum(a, dtype=float))[::-1], c=colors[i], label = 'file') 
plt.legend(fontsize=16)
plt.show()

## Zipf per cancer

In [ ]:
colors = ['red', 'blue', 'yellow', 'green']
fig = plt.figure(figsize=(8,6))
x = np.logspace(0, 5)
plt.ylabel("$frequency_i$", fontsize=16)
plt.xlabel("$RANK_i$", fontsize=16)
plt.yscale('log')
plt.xscale('log')
plt.plot(x, 1./x, 'g--', label='$r^{-1}$')
for i,k in enumerate(df.keys()[1:]):
    a=df[k]
    plt.plot(np.sort(a/np.sum(a))[::-1], c=colors[i], label = k) 
plt.legend(fontsize=16)
plt.show()

In [ ]:
fig.savefig("zipfPerCancer.pdf")

# gene table

In [ ]:
genes = open("%s/genes.txt"%working_dir, "w")
for gene in cdf['gene'].values:
    genes.write("%s\n"%gene)
genes.close()

# table as image

In [ ]:
from PIL import Image

In [ ]:
bits = np.array([df.loc[i,:].values[1:] for i in np.arange(len(df['gene']))])

In [ ]:
bits.shape

In [ ]:
img = Image.fromarray(bits, mode='1')

In [ ]:
img.save("mat.bmp")